In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [3]:
from operator import itemgetter

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS

In [4]:
vectorstore = FAISS.from_texts(
    ["Shiva studied at UMD, Go Bulldogs!", "Shiva grew up in Hyderabad, India", "Shiva works at Microsoft"],
    embedding=OpenAIEmbeddings(chunk_size=10)
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

In [5]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("where did Shiva grow up?")

'Shiva grew up in Hyderabad, India.'

In [6]:
chain.invoke("What is Shiva's birthplace?")

"Shiva's birthplace is Hyderabad, India."

In [17]:
# Understanding itemgetter -  a handy tool for extracting specific values from iterables, making it easier to work with and manipulate them.

from operator import itemgetter

people = [('Alice', 30, 5.5), ('Bob', 25, 6.0), ('Charlie', 35, 5.9)]

# Use itemgetter to get the second element (age) from each tuple and create get_age function
get_age = itemgetter(1)

# Now you can use get_age as a key function for sorting
sorted_people = sorted(people, key=get_age)

print(sorted_people)

for p in people:
    print(get_age(p))


[('Bob', 25, 6.0), ('Alice', 30, 5.5), ('Charlie', 35, 5.9)]
30
25
35


In [18]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
chain.invoke({"question": "where did shiva work", "language": "hindi"})

'शिवा माइक्रोसॉफ्ट में काम करते हैं।'

# Conversational Retrieval Chain

In [23]:
from langchain.schema import format_document
from langchain.schema.runnable import RunnableMap

from langchain.prompts.prompt import PromptTemplate

condense_question_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_question_template)

In [24]:
answer_template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

In [25]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [26]:
from typing import List, Tuple

def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

In [27]:
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | model

In [31]:
response = conversational_qa_chain.invoke(
    {
        "question": "where did Shiva work?",
        "chat_history": [],
    }
)
response

AIMessage(content='Shiva was employed at Microsoft.')

In [59]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
loaded_memory

RunnableAssign(mapper={
  chat_history: RunnableLambda(...)
                | RunnableLambda(...)
})

In [74]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory(
#     return_messages=True, output_key="answer", input_key="question"
# )

memory = ConversationBufferMemory(return_messages=True)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)

# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | model,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [75]:
inputs = {"question": "where did Shiva work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Shiva was employed at Microsoft.'),
 'docs': [Document(page_content='Shiva works at Microsoft'),
  Document(page_content='Shiva grew up in Hyderabad, India'),
  Document(page_content='Shiva studied at UMD, Go Bulldogs!')]}

In [76]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did Shiva work?'),
  AIMessage(content='Shiva was employed at Microsoft.')]}

In [77]:
memory.load_memory_variables

<bound method ConversationBufferMemory.load_memory_variables of ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='where did Shiva work?'), AIMessage(content='Shiva was employed at Microsoft.')]), return_messages=True)>

In [80]:
inputs = {"question": "who is the current ceo of that company?"}
result = final_chain.invoke(inputs)
result

TypeError: 'HumanMessage' object is not subscriptable